In [2]:
import uproot
import awkward as ak 
import vector
vector.register_awkward()
import matplotlib.pyplot as plt
import os 


In [26]:

with uproot.open("/vols/cms/emc21/idmStudy/BPs/MadGraph_files/paramScan/trilepton/lam345scan/h2h2lllnu/h2h2lllnu_BP2_lam345_-1/Events/run_01/unweighted_events.root:LHEF;1") as f:
    weights = ak.to_numpy(f['Event']['Event.Weight'].arrays()['Event.Weight'])
    print(weights)
    tree = f['Particle']
    events = tree.arrays(library="ak", how="zip")

part = events.Particle
# Only want final state particles 
part = part[part['Status'] == 1]
branches = ['Px', 'Py', 'Pz', 'E', 'M', 'PT', 'Eta', 'Phi', 'Rapidity']
for b in branches:
    part[b.lower()] = part[b]


part = ak.Array(part, with_name="Momentum4D")
# print(ak.flatten(weights['Event.Weight']))
part['weights'] = weights


[[0.0033305]
 [0.0033305]
 [0.0033305]
 ...
 [0.0033305]
 [0.0033305]
 [0.0033305]]


In [28]:
print(part[0].weights)

[0.00333, 0.00333, 0.00333, 0.00333, 0.00333, 0.00333]


In [19]:
print(ak.to_list(part[0]))

[{'fUniqueID': 0, 'fBits': 50331648, 'PID': 35, 'Status': 1, 'Mother1': 2, 'Mother2': 2, 'ColorLine1': 0, 'ColorLine2': 0, 'Px': -42.721086044, 'Py': -113.08168485, 'Pz': 217.68174013, 'E': 257.33810178, 'M': 65.0, 'PT': 120.88241659271034, 'Eta': 1.3508158724584811, 'Phi': -1.9320104400204265, 'Rapidity': 1.2415525976241863, 'LifeTime': 0.0, 'Spin': 0.0, 'px': -42.721086044, 'py': -113.08168485, 'pz': 217.68174013, 'e': 257.33810178, 'm': 65.0, 'pt': 120.88241659271034, 'eta': 1.3508158724584811, 'phi': -1.9320104400204265, 'rapidity': 1.2415525976241863, 'weights': 0.0033305009}, {'fUniqueID': 0, 'fBits': 50331648, 'PID': 35, 'Status': 1, 'Mother1': 3, 'Mother2': 3, 'ColorLine1': 0, 'ColorLine2': 0, 'Px': 31.326510314, 'Py': 121.61884132, 'Pz': 131.27234973, 'E': 192.9505704, 'M': 65.0, 'PT': 125.58858551824082, 'Eta': 0.9130158288253074, 'Phi': 1.318696913841394, 'Rapidity': 0.829750211613922, 'LifeTime': 0.0, 'Spin': 0.0, 'px': 31.326510314, 'py': 121.61884132, 'pz': 131.27234973, 

In [10]:
lep = part[(abs(part.PID) == 11) | (abs(part.PID) == 13)]

dimu_events = lep[ak.num(lep[abs(lep.PID) == 13]) == 2]
dimu = dimu_events[abs(dimu_events['PID']) == 13]
dimu_extra_lep = dimu_events[abs(dimu_events['PID']) == 11]

dielec_events = lep[ak.num(lep[abs(lep.PID) == 11]) == 2]
dielec = dielec_events[abs(dielec_events['PID']) == 11]
dielec_extra_lep = dielec_events[abs(dielec_events['PID']) == 13]

dilep = ak.concatenate([dimu, dielec], axis=0)
lep = ak.concatenate([dimu_extra_lep, dielec_extra_lep], axis=0)


In [11]:
dilep1, dilep2 = dilep[:,0], dilep[:,1]
dilep4v = dilep1 + dilep2
dilepmass = dilep4v.mass
dilepdR = dilep1.deltaR(dilep2)
dilepPT = dilep4v.pt
singlePT = ak.flatten(lep.pt)
dilepdRlep = ak.flatten(dilep4v.deltaR(lep))


dm = part[part.PID == 35]
# Get vector sum to find the MET 
dm1 = dm[:,0]
dm2 = dm[:,1]
MET = dm1 + dm2
print(len(MET))
print(dm.weights)

10000
[[0.00333, 0.00333], [0.00333, 0.00333], [...], ..., [...], [0.00333, 0.00333]]


In [13]:
print(singlePT)
print(dilepdRlep)
print(weights)

[22.2, 24.1, 31.1, 64.3, 20.2, 28.8, ..., 18.6, 32.5, 48, 1.37, 27.4, 2.95]
[2.65, 1.47, 3.11, 2.47, 1.92, 2.76, ..., 0.607, 2.87, 2.68, 3.35, 3.01, 1.95]
[0.00333, 0.00333, 0.00333, 0.00333, ..., 0.00333, 0.00333, 0.00333, 0.00333]


In [11]:
print(singlePT)
print(dilepdRlep)
print(weights)

[[22.2], [24.1], [31.1], [64.3], [20.2], ..., [48], [1.37], [27.4], [2.95]]
[[2.65], [1.47], [3.11], [2.47], [1.92], ..., [2.68], [3.35], [3.01], [1.95]]
[[0.00333], [0.00333], [0.00333], ..., [0.00333], [0.00333], [0.00333]]


In [32]:
arr = ak.Array({"x" : [[1, 2, 1], [1], [4, 5, 6, 7]],
                "y" : [[1], [5, 6, 7], [3, 4, 5, 6]]})
# weights = ak.Array({"weights" : [[15], [16], [17]]})
weights = np.array([[15], [16], [17]])
arr['weights'] = weights
arr

<Array [{x: [1, ..., 1], y: [1], ...}, ...] type='3 * {x: var * int64, y: v...'>

In [34]:
part[0]

<MomentumArray4D [{fUniqueID: 0, fBits: ..., ...}, ...] type='6 * Momentum4...'>